Topologies are modeled in terms of the relative bandwidths of the links. We assume that all per-send latencies are uniform, which is mostly true over NVLinks.

In [24]:
from msccl.topologies import fully_connected
from pprint import pprint
num_nodes = 8

Make a fully connected topology.

In [38]:
topology_1 = fully_connected(num_nodes)
pprint(topology_1.links)

[[0, 1, 1, 1, 1, 1, 1, 1],
 [1, 0, 1, 1, 1, 1, 1, 1],
 [1, 1, 0, 1, 1, 1, 1, 1],
 [1, 1, 1, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 0, 1, 1, 1],
 [1, 1, 1, 1, 1, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 1],
 [1, 1, 1, 1, 1, 1, 1, 0]]


The collective is the specification for where chunks start at and where they need to go. Here we instantiate allgather for this topology.

In [39]:
from msccl.collectives import allgather, alltoall, reduce_scatter, allreduce
collective = allgather(num_nodes)

Here is the precondition. We can see that all ranks start with one chunk.

In [40]:
pprint([[1 if collective.precondition(rank, chunk) else 0 for chunk in range(collective.num_chunks)] for rank in range(collective.num_nodes)])

[[1, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 1]]


Here is the postcondition. All ranks need to get all chunks.

In [41]:
pprint([[1 if collective.postcondition(rank, chunk) else 0 for chunk in range(collective.num_chunks)] for rank in range(collective.num_nodes)])

[[1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1]]


Solve latency-bandwidth tradeoffs for both topologies.

In [42]:
from msccl.strategies import solve_all_latency_bandwidth_tradeoffs
algos_1 = list(solve_all_latency_bandwidth_tradeoffs(topology_1, collective, logging=True))

Algorithms need at least 1 steps.
Algorithms need at least 1 rounds per chunk.
Solving instance steps=1... synthesized! (0.2s)
Bandwidth optimal algorithm found!


Two preprocessing steps are performed:
- The minimum number of steps required is lower bound based on the maximum of the shortest paths for each chunk considering the topology.
- A minimum number of rounds per chunk is lower bound using a kind of multi-commodity flow encoding in [rounds_bound.py](../msccl/rounds_bound.py).

Then all relevant trade-offs are iterated until a bandwidth optimal algorithm is found (if the rounds per chunk lower bound happens to be exact).

The synthesized algorithms contain many non-Pareto-optimal algorithms, which are dominated by some other algorithm for all input sizes. We can filter those out:

In [43]:
from msccl.strategies import prune_pareto_optimal
algos_1 = prune_pareto_optimal(algos_1)

Lets set up a function to analyze the performance of the remaining algorithms. Here we assume that alpha=1 and beta=1.

In [44]:
from fractions import Fraction
def print_perf(size, algos):
    print(f'Input size is {size}')
    for algo in algos:
        print(f'\n{algo.name}')
        chunk_size = Fraction(1, algo.instance.chunks)
        print(f'Chunk size:             1/chunks = {chunk_size} = {float(chunk_size)}')
        bw_mult = algo.instance.rounds() * chunk_size
        print(f'BW multiples:      rounds/chunks = {bw_mult} = {float(bw_mult)}')
        time = algo.instance.steps + size * bw_mult
        print(f'Time: steps + size*rounds/chunks = {time}')

See a rough estimate of performance for a big input.

In [48]:
print_perf(10.0, algos_1)

Input size is 10.0

Allgather(n=8)-FullyConnected(n=8)-steps=1
Chunk size:             1/chunks = 1 = 1.0
BW multiples:      rounds/chunks = 1 = 1.0
Time: steps + size*rounds/chunks = 11.0


See a rough estimate of performance for a small input.

In [49]:
print_perf(0.1, algos_1)

Input size is 0.1

Allgather(n=8)-FullyConnected(n=8)-steps=1
Chunk size:             1/chunks = 1 = 1.0
BW multiples:      rounds/chunks = 1 = 1.0
Time: steps + size*rounds/chunks = 1.1


Export the algorithm to xml format for loading into msccl.

In [37]:
from msccl.ncclize import ncclize

algo_name = f'topology_allgather_{num_nodes}.xml'
with open(algo_name, 'w') as f:
    f.write(ncclize(algos_1[0]))

Use this command to run the xml files:
```
mpirun -x MSCCL_XML_FILES=/test.xml --allow-run-as-root -np 8 -x LD_LIBRARY_PATH=/msccl/build/lib/:$LD_LIBRARY_PATH -x NCCL_DEBUG=WARN -x NCCL_DEBUG_SUBSYS=INIT,ENV -x NCCL_ALGO=MSCCL,RING,TREE /nccl-tests/build/all_reduce_perf -b 128 -e 8GB -f 2 -g 1 -c 0 -n 10 -w 10 -G 10 -z 0 
```